In [2]:
from FCI_MATRIX_helper import *
from STATE_PREP_helper import *

In [3]:
lambda_value = 0.0001
lambda_min = -lambda_value
lambda_max = lambda_value
nuclear_dipole_moment = 35.290635377386
au = 2.54177000
# 1 Debye =   2.54177000 a.u. 
#integral reading after applying perturbation -0.001 
file_path = f"C:/Users/prita/Desktop/DIRAC_FF_DATA/DIRAC_FF_DATA/SrF_finitefield_DATA/SrF_FF_{lambda_value}/-{lambda_value}/FCIDUMP"
# C:\Users\prita\Desktop\DIRAC_FF_DATA\DIRAC_FF_DATA\SrF_finitefield_DATA\SrF_FF_0.001\-0.001\FCIDUMP
integral_dict_min0001 = read_integrals(file_path)

#integral reading after applying perturbation +0.001 
file_path = f"C:/Users/prita/Desktop/DIRAC_FF_DATA/DIRAC_FF_DATA/SrF_finitefield_DATA/SrF_FF_{lambda_value}/+{lambda_value}/FCIDUMP"
# C:\Users\prita\Desktop\DIRAC_FF_DATA\DIRAC_FF_DATA\SrF_finitefield_DATA\SrF_FF_0.001\-0.001\FCIDUMP
integral_dict_max0001 = read_integrals(file_path)

In [4]:
N_ORB = 8
N_ELEC = 3
N_alpha = 2
N_beta = 1

#State preparation for the perturbation -0.001 and +0.001
states  = generate_states(N_ORB, N_ELEC, N_alpha, N_beta)
print("Number of generated states:", len(states))
print("Generated states:")
for state in states:
    print(f'{states[0]} | {state} => Nexcite = {len(set(state) - set(states[0]))}')
    # discard the states with more than 2 excitations
    if len(set(state) - set(states[0])) > 2:
        states.remove(state)
print("Number of states after removing states with more than 2 excitations:", len(states))


Number of generated states: 24
Generated states:
[1, 2, 3] | [1, 2, 3] => Nexcite = 0
[1, 2, 3] | [1, 2, 5] => Nexcite = 1
[1, 2, 3] | [1, 2, 7] => Nexcite = 1
[1, 2, 3] | [1, 3, 4] => Nexcite = 1
[1, 2, 3] | [1, 3, 6] => Nexcite = 1
[1, 2, 3] | [1, 3, 8] => Nexcite = 1
[1, 2, 3] | [1, 4, 5] => Nexcite = 2
[1, 2, 3] | [1, 4, 7] => Nexcite = 2
[1, 2, 3] | [1, 5, 6] => Nexcite = 2
[1, 2, 3] | [1, 5, 8] => Nexcite = 2
[1, 2, 3] | [1, 6, 7] => Nexcite = 2
[1, 2, 3] | [1, 7, 8] => Nexcite = 2
[1, 2, 3] | [2, 3, 5] => Nexcite = 1
[1, 2, 3] | [2, 3, 7] => Nexcite = 1
[1, 2, 3] | [2, 5, 7] => Nexcite = 2
[1, 2, 3] | [3, 4, 5] => Nexcite = 2
[1, 2, 3] | [3, 4, 7] => Nexcite = 2
[1, 2, 3] | [3, 5, 6] => Nexcite = 2
[1, 2, 3] | [3, 5, 8] => Nexcite = 2
[1, 2, 3] | [3, 6, 7] => Nexcite = 2
[1, 2, 3] | [3, 7, 8] => Nexcite = 2
[1, 2, 3] | [4, 5, 7] => Nexcite = 3
[1, 2, 3] | [5, 7, 8] => Nexcite = 3
Number of states after removing states with more than 2 excitations: 22


In [9]:
hf_energy_min001 = Matrix_elements(states[0], states[0], integral_dict_min0001)
print(f"HF energy for lambda=-{lambda_value}:", hf_energy_min001 + integral_dict_min0001["(0, 0, 0, 0)"])
E_min = hf_energy_min001 + integral_dict_min0001["(0, 0, 0, 0)"]

HF energy for lambda=-0.0001: -3241.560590100807


In [10]:
hf_energy_max001 = Matrix_elements(states[0], states[0], integral_dict_max0001)
print(f"HF energy for lambda=+{lambda_value}:", hf_energy_max001 + integral_dict_max0001["(0, 0, 0, 0)"])
E_max = hf_energy_max001 + integral_dict_max0001["(0, 0, 0, 0)"]

HF energy for lambda=+0.0001: -3241.553380787281


In [11]:
#Finite field calculation :
elec_dipole_moment  = abs(E_max - E_min) / abs(lambda_max - lambda_min)
print("Dipole moment:", elec_dipole_moment)

Dipole moment: 36.04656762945524


In [12]:
print("Results from SCF energies : ")
dm_db = abs(nuclear_dipole_moment-elec_dipole_moment)
print("Dipole moment in debye:", dm_db)
print("Dipole moment in a.u.:", dm_db*au)


Results from SCF energies : 
Dipole moment in debye: 0.7559322520692362
Dipole moment in a.u.: 1.9214059203420226


In [13]:
#Calculation from FCI energies
FCI_min0001 = FCI_matrix(states, integral_dict_min0001)
FCI_max0001 = FCI_matrix(states, integral_dict_max0001)

e_FCI_min0001 = FCI_eign(FCI_min0001)
e_FCI_max0001 = FCI_eign(FCI_max0001)

In [14]:
#Finite field calculation :
print("Results from FCI energies :")
FCI_energy_min001 = e_FCI_min0001 + integral_dict_min0001["(0, 0, 0, 0)"]
FCI_energy_max001 = e_FCI_max0001 + integral_dict_max0001["(0, 0, 0, 0)"]
elec_dipole_moment_FCI  = abs(FCI_energy_max001[0] - FCI_energy_min001[0]) / abs(lambda_max - lambda_min)
print("Dipole moment from FCI energies:", elec_dipole_moment_FCI)
dm_db = abs(nuclear_dipole_moment-elec_dipole_moment)
print("Dipole moment in debye:", dm_db)
print("Dipole moment in a.u.:", dm_db*au)


Results from FCI energies :
Dipole moment from FCI energies: 35.795216540464025
Dipole moment in debye: 0.7559322520692362
Dipole moment in a.u.: 1.9214059203420226
